In [27]:
import os
import random
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import f1_score
from imblearn.over_sampling import RandomOverSampler
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE


device(type='cuda')

In [28]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

EPOCHS = 35
BATCH_SIZE = 512
LR = 1e-3
WEIGHT_DECAY = 1e-4
PATIENCE = 5 
NUM_FOLDS = 10

drop_cols = ["quality_flag",]

In [29]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# one-hot цели из type
type_dummies = pd.get_dummies(train['type'], prefix='type')
train = pd.concat([train, type_dummies], axis=1)
train = train.drop('type', axis=1)
TARGET_COLS = [col for col in train.columns if col.startswith('type_')]

qcats = sorted(set(train['quality_flag'].astype(str).unique()).union(set(test['quality_flag'].astype(str).unique())))
for c in qcats:
    col = f'quality_flag_{c}'
    train[col] = (train['quality_flag'].astype(str) == c).astype(int)
    test[col] = (test['quality_flag'].astype(str) == c).astype(int)

train = train.drop(columns=drop_cols)
test = test.drop(columns=drop_cols)

X_cols = [c for c in train.columns if c not in TARGET_COLS]# + ['idx']]
X = train[X_cols].copy()
y = np.argmax(train[TARGET_COLS].values, axis=1)
class_names = [c.replace('type_', '') for c in TARGET_COLS]
num_classes = len(TARGET_COLS)
X.shape, num_classes


((45000, 29), 6)

In [ ]:
class TabularDataset(Dataset):
    def __init__(self, features: np.ndarray, targets: np.ndarray | None = None):
        self.features = features.astype(np.float32)
        self.targets = None if targets is None else targets.astype(np.int64)
    def __len__(self):
        return len(self.features)
    def __getitem__(self, idx):
        if self.targets is None:
            return torch.from_numpy(self.features[idx])
        return torch.from_numpy(self.features[idx]), torch.tensor(self.targets[idx])
        
class MLP(nn.Module):
    def __init__(self, in_dim: int, out_dim: int):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.Linear(128, out_dim),
        )
    def forward(self, x):
        return self.net(x)

X_np = X.values.copy()

X_test_np = test[X_cols].values.copy()

X_tr, y_tr = (X_np, y)
transform = StandartScaler() 
X_tr_scaled = transform.fit_transform(X_tr)
X_test_scaled = transform.transform(X_test_np)

val_size = 5000
indices = np.arange(len(X_tr_scaled))
np.random.seed(SEED)
np.random.shuffle(indices)

val_indices = indices[:val_size]
train_indices = indices[val_size:]

X_train_final = X_tr_scaled[train_indices]
y_train_final = y_tr[train_indices]

ros = RandomOverSampler(random_state=SEED)
X_train_final, y_train_final = ros.fit_resample(X_train_final, y_train_final)

X_val = X_tr_scaled[val_indices]
y_val = y_tr[val_indices]

train_ds = TabularDataset(X_train_final, y_train_final)
val_ds = TabularDataset(X_val, y_val)
test_ds = TabularDataset(X_test_scaled, None)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)

model = MLP(in_dim=X_tr.shape[1], out_dim=num_classes).to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
criterion = nn.CrossEntropyLoss()

best_f1 = -1.0
best_state = None
epochs_no_improve = 0
for epoch in range(1, EPOCHS + 1):
    model.train()
    total_loss = 0.0
    for xb, yb in train_loader:
        xb = xb.to(DEVICE)
        yb = yb.to(DEVICE)
        optimizer.zero_grad()
        logits = model(xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * xb.size(0)
    
    model.eval()
    val_preds = []
    val_targets = []
    with torch.no_grad():
        for xb, yb in val_loader:
            xb = xb.to(DEVICE)
            logits = model(xb)
            preds = logits.argmax(dim=1).detach().cpu().numpy()
            val_preds.append(preds)
            val_targets.append(yb.numpy())
    val_preds = np.concatenate(val_preds)
    val_targets = np.concatenate(val_targets)
    f1 = f1_score(val_targets, val_preds, average='macro')
    
    if f1 > best_f1:
        best_f1 = f1
        best_state = {k: v.cpu() for k, v in model.state_dict().items()}
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
    
    print(f'Epoch {epoch}: val F1 macro = {f1:.4f}, best = {best_f1:.4f}')
    
    if epochs_no_improve >= PATIENCE:
        print(f'Early stopping на эпохе {epoch}')
        break

    if f1 > 0.999:
        break 

if best_state is not None:
    model.load_state_dict({k: v.to(DEVICE) for k, v in best_state.items()})
model.eval()
test_logits = []
with torch.no_grad():
    for xb in test_loader:
        xb = xb.to(DEVICE)
        logits = model(xb)
        test_logits.append(logits.detach().cpu().numpy())
test_logits = np.concatenate(test_logits)

Epoch 1: val F1 macro = 0.3956, best = 0.3956
Epoch 2: val F1 macro = 0.6816, best = 0.6816
Epoch 3: val F1 macro = 0.7475, best = 0.7475
Epoch 4: val F1 macro = 0.8056, best = 0.8056
Epoch 5: val F1 macro = 0.7551, best = 0.8056
Epoch 6: val F1 macro = 0.8151, best = 0.8151
Epoch 7: val F1 macro = 0.8330, best = 0.8330
Epoch 8: val F1 macro = 0.8338, best = 0.8338
Epoch 9: val F1 macro = 0.8266, best = 0.8338
Epoch 10: val F1 macro = 0.8080, best = 0.8338
Epoch 11: val F1 macro = 0.7914, best = 0.8338
Epoch 12: val F1 macro = 0.8352, best = 0.8352
Epoch 13: val F1 macro = 0.8413, best = 0.8413
Epoch 14: val F1 macro = 0.7623, best = 0.8413
Epoch 15: val F1 macro = 0.8050, best = 0.8413
Epoch 16: val F1 macro = 0.8435, best = 0.8435
Epoch 17: val F1 macro = 0.8384, best = 0.8435
Epoch 18: val F1 macro = 0.8474, best = 0.8474
Epoch 19: val F1 macro = 0.8528, best = 0.8528
Epoch 20: val F1 macro = 0.8575, best = 0.8575
Epoch 21: val F1 macro = 0.8340, best = 0.8575
Epoch 22: val F1 macro

In [31]:
# all features - 0.985 
# drop quality flag - 0.9864
# drop r_mag - 0.9866 
# drop i_mag - 0.9918
# drop pm_ra - 0.9933 
# 

In [32]:
# формируем submission (жесткая классификация)
probs = torch.softmax(torch.from_numpy(test_logits), dim=1).numpy()
preds = probs.argmax(axis=1)

submission = pd.DataFrame(0, index=np.arange(len(test)), columns=TARGET_COLS)
for i, cls_idx in enumerate(preds):
    submission.iloc[i, cls_idx] = 1

submission['idx'] = test['idx'].values
submission = submission[['idx'] + TARGET_COLS]
submission.columns = ['idx'] + [c.replace('type_', '') for c in TARGET_COLS]
submission.to_csv('submission_nn.csv', index=False)

print('submission.csv saved')


submission.csv saved


In [33]:
# Baseline: 0.9921

In [34]:
list_submits = [f'submission_nn{i}.csv' for i in range(1, 6)]

dfs = [pd.read_csv(f) for f in list_submits]
idx = dfs[0]['idx'].values
class_cols = [c for c in dfs[0].columns if c != 'idx']

votes = np.zeros((len(idx), len(class_cols)))
for df in dfs:
    for i, col in enumerate(class_cols):
        votes[:, i] += df[col].values

final_preds = votes.argmax(axis=1)

submission = pd.DataFrame(0, index=np.arange(len(idx)), columns=class_cols)
for i, cls_idx in enumerate(final_preds):
    submission.iloc[i, cls_idx] = 1

submission['idx'] = idx
submission = submission[['idx'] + class_cols]
submission.to_csv('submission_ensemble.csv', index=False)

print('submission_ensemble.csv saved')

submission_ensemble.csv saved


In [35]:
list_submits

['submission_nn1.csv',
 'submission_nn2.csv',
 'submission_nn3.csv',
 'submission_nn4.csv',
 'submission_nn5.csv']